In [1]:
import sys
from bs4 import BeautifulSoup
import requests
import datetime as dt
from PIL import Image
import helpers as hf

import yaml
from unidecode import unidecode
import pandas as pd
from PIL import Image, ImageOps, ImageDraw, ImageFont
import json
import numpy as np


with open("info/design_config.yml", "r") as f:
    design_config = yaml.safe_load(f)

design_config,shop_config = hf.get_config('cbb')
cfbd_loader = hf.cfbp_handler(design_config['cfbd_api'])

private file size: 2466


In [5]:
headers = {
            "X-Shopify-Access-Token": shop_config["shopify_access"],
            "Content-Type": "application/json",
        }

In [153]:
response = requests.get(products_link, headers=headers)
products = response.json()["products"]
print(len(products))

85


In [161]:
products_link = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2024-01/products.json?limit=250"

def recur_get_products(products_link,products=[]):
    response = requests.get(products_link, headers=headers)
    products.extend(response.json()["products"])
    if 'next' in response.links.keys():
        # theres another page of products
        return recur_get_products(response.links['next']['url'],products=products)
    else:
        return products


In [164]:
type(np.nan)

float

In [162]:
d = recur_get_products(products_link)

again
again


In [163]:
len(d)

85

In [157]:
while 'next' in response.links.keys():


False

In [154]:
next_link = 
response2 = requests.get(next_link, headers=headers)
products2 = response2.json()["products"]
print(len(products2))

KeyError: 'next'

In [152]:
products[0]

{'id': 8350558486749,
 'title': 'Alabama Vs San Diego State. 24 Mar, 2023 Post-Game',
 'body_html': "AI image representing the basketball game 64 to 71 between Alabama and San Diego State, played at  24 Mar, 2023\n||Alabama|| ||San Diego State||ROUND: ||NCAA Men's Tournament, South - Regional Semifinal||",
 'vendor': 'Printify',
 'product_type': 'T-Shirt',
 'created_at': '2024-01-28T12:40:50-08:00',
 'handle': 'alabama-vs-san-diego-state-24-mar-2023-post-game',
 'updated_at': '2024-02-03T14:34:38-08:00',
 'published_at': '2024-01-28T12:40:52-08:00',
 'template_suffix': None,
 'published_scope': 'web',
 'tags': 'Alabama, basketball, college, madness, march, marchmadness, San Diego State, sports',
 'status': 'active',
 'admin_graphql_api_id': 'gid://shopify/Product/8350558486749',
 'variants': [{'product_id': 8350558486749,
   'id': 44510928830685,
   'title': 'Sport Grey / S',
   'price': '25.99',
   'sku': '13742892631291301772',
   'position': 1,
   'inventory_policy': 'deny',
   'com

In [136]:
response.url

'https://c2b32f-2.myshopify.com/admin/api/2024-01/products.json?limit=250'

In [105]:
response.url

'https://c2b32f-2.myshopify.com/admin/api/2024-01/products.json?limit=250'

In [133]:
response.headers["has_next_page"]

KeyError: 'has_next_page'

In [124]:
dict(response.headers)["X-Shopify-API-Version"]

KeyError: 'X-Shopify-API-Version'

In [13]:
for p in products:
    if 'memphis' in p['title'].lower():
        print(p)

{'id': 8366289092829, 'title': 'Memphis Vs Smu. 07 Jan, 2024', 'body_html': 'AI image representing the basketball game 62 to 59 between Memphis and SMU, played at  07 Jan, 2024Memphis Tigers basketball shirt SMU Mustangs basketball shirt\n||Memphis|| ||SMU||ROUND: ||None||', 'vendor': 'Printify', 'product_type': 'T-Shirt', 'created_at': '2024-02-16T08:25:44-08:00', 'handle': 'memphis-vs-smu-07-jan-2024', 'updated_at': '2024-02-16T08:28:37-08:00', 'published_at': '2024-02-16T08:25:46-08:00', 'template_suffix': None, 'published_scope': 'web', 'tags': 'basketball, college, madness, march, marchmadness, Memphis, Memphisbasketball, SMUSMU, sports', 'status': 'active', 'admin_graphql_api_id': 'gid://shopify/Product/8366289092829', 'variants': [{'id': 44566476357853, 'product_id': 8366289092829, 'title': 'Sport Grey / S', 'price': '25.99', 'sku': '10337885663125397649', 'position': 1, 'inventory_policy': 'deny', 'compare_at_price': None, 'fulfillment_service': 'printify', 'inventory_managemen

In [66]:
url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2021-07/custom_collections.json"

# Make the API request to get the list of collections
response = requests.get(url, headers=headers)

In [67]:
response.json()['custom_collections']

[]

In [14]:
# testing parenthesis in PIL
import PIL
from PIL import Image, ImageFont, ImageDraw

im1=Image.open('art/cover3.jpeg')

# Drawing the text on the picture

font = ImageFont.truetype('fonts/Freshman.ttf', 100)
draw = ImageDraw.Draw(im1)
draw.text((50, 200), 'test(1)', (255,0,255), font=font)

# Save the image with a new name
im1.show()

34

In [65]:
text = Image.new("RGBA", (450, 510),color='grey')

text_draw = ImageDraw.Draw(text)


t_text = 'University of Texas of the Permian Basin vs Southwestern Assemblies of God'

game_font = hf.simple_size_font_recur(
    text_draw, t_text, 450, "Freshman", 50
)
print(game_font.size)
if game_font.size<24:
    # when font gets this small, stroke makes hard to read
    main_stroke = 0
    
text_draw.text((0, 0), t_text, font=game_font, fill="white",stroke_width=main_stroke,stroke_fill="black")
text.show()


10


In [50]:
teams = cfbd_loader.get_team_info()

In [52]:
teams['name_l'] = teams.name.apply(lambda x:len(x))

In [63]:
teams.loc[teams.name_l>29]

,name,abrev,alt_color,color,division,id,logos,long_name,mascot,team_id,name_l
197,Brooklyn Polytechnic Institute,NaN,None,#FFFFFF,None,1000080.0,NaN,None,NaN,NaN,30
754,Jacksonville Naval Air Station,NaN,None,#FFFFFF,None,1000031.0,NaN,None,Flyers,NaN,30
1447,Southwestern Assemblies of God,SWA,None,#000000,None,2904.0,http://a.espncdn.com/i/teamlogos/ncaa/500/2904...,None,Lions,NaN,30
1649,University of Texas of the Permian Basin,UTPB,None,#FFFFFF,ii,110243.0,NaN,None,Falcons,NaN,40
